In [1]:
import numpy as np

from db_handler import DBHandler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

In [2]:
db = DBHandler()
db.setup_test_table()

highest = 0
user_info = {}

for x in db.read_data("comp3208-train-small.csv"):
    listParts = x.strip().split(',')
    if listParts[2] == "rating":
        continue
    if int(listParts[1]) > highest:
        highest = int(listParts[1])
    if listParts[0] in user_info:
        user_info[listParts[0]][listParts[1]] = float(listParts[2])
    else:
        user_info[listParts[0]] = {listParts[1]: float(listParts[2])}

INFO 2021-02-25 15:12:01,248 logging started
INFO 2021-02-25 15:12:01,289 Setting up test table
INFO 2021-02-25 15:12:01,300 Reading data from database: comp3208-train-small.csv


In [3]:
matrix = np.zeros((len(user_info), (highest + 1)), dtype="float32")
count = 0

for user, info in user_info.items():
    row = [0] * (highest + 1)
    for item, rating in info.items():
        row[int(item)] = rating
    matrix[count] = row
    count += 1

In [53]:
db = DBHandler()
db.setup_test_table()

row = [] # user
col = [] # item
rating = []

for x in db.read_data("comp3208-train-small.csv"):
    line_split = x.strip().split(',')
    if line_split[2] == "rating":
        continue
    row.append(int(line_split[0]) - 1)
    col.append(int(line_split[1]) - 1)
    rating.append(float(line_split[2]))


INFO 2021-02-25 16:26:19,789 logging started
INFO 2021-02-25 16:26:19,794 Setting up test table
INFO 2021-02-25 16:26:20,847 Reading data from database: comp3208-train-small.csv


In [54]:
R = coo_matrix((rating, (row, col)), shape=((max(row) + 1), (max(col) + 1)))

# Latent Factor Model

In [65]:
from numpy.linalg import norm

def calc_mse(R, U, V, lamda=0.02):
    ratings = R.data
    rows = R.row
    cols = R.col
    total_error = 0
    
    for index in range(len(ratings)):
        rating = ratings[index]
        user = rows[index]
        item = cols[index]
        
        # lambda for regularisation
        if rating > 0:
            total_error = total_error + pow(rating - np.dot(U[user,:], V[:,item]), 2) + lamda * (pow(norm(U[user,:]), 2) + pow(norm(V[:,item]), 2))
    
    return total_error / len(ratings)

In [67]:
def gradient_descent(R, K, lam = 0.02, steps = 10, learning_rate = 0.001):
    # using K create matricies U and V
    M, N = R.shape
    U = np.random.rand(M, K)
    V = np.random.rand(K, N)
    mse = calc_mse(R, U, V)
    
    for step in range(steps):
        for index in range(len(R.data)):
            # --> you can parallelise this
            rating = R.data[index]
            user = R.row[index]
            item = R.col[index]
            # ignore 0 ratings as its means that no rating has been made
            if rating > 0:
                loss = rating - np.dot(U[user,:], V[:,item])
                U[user,:] = U[user,:] + learning_rate * 2 * (loss * V[:,item] - lam * U[user,:])
                V[:,item] = V[:,item] + learning_rate * 2 * (loss * U[user,:] - lam * V[:,item])
            # <-- end
        mse = calc_mse(R, U, V)
    print("Final MSE:", mse)
    
    return U, V

In [68]:
U, V = gradient_descent(R, K=3, gamma=0.0007, lam=0.01, steps=10)

MemoryError: Unable to allocate 42.2 GiB for an array with shape (283229, 19999) and data type float64

# Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=5, random_state=0)

tree.fit(X_train, y_train)

prediction = tree.predict(X_test)

In [8]:
print(mean_squared_error(y_test.astype(np.float64), prediction.astype(np.float64)))

1.3974708535230393


# SVC

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0, tol=1e-5)
svc.fit(X_train, y_train)

prediction = svc.predict(X_test)

In [ ]:
print(mean_squared_error(y_test.astype(np.float64), prediction))